Get high KaKs region in python

In [25]:
from tqdm.contrib import tenumerate
import numpy as np
import pandas as pd
from functools import partial

In [28]:
# Define helper function
def get_KaKs_region(geneid_df, threshold=1, step_size=6, block_length=30):
    vec = (geneid_df['Ka/Ks'] > threshold).to_numpy()
    
    n_bases = 0
    KaKs_r = 0
    n_blocks = 0
    pos_info = "pos"
    
    if vec.sum() == 0:
        return n_bases, KaKs_r, n_blocks, pos_info
    
    n_bases = vec.sum() * step_size
    KaKs_r = vec.sum() / len(vec)
    
    for start in range(len(vec)):
        if vec[start]:
            for end in range(start, len(vec)):
                length = end - start + 1
                if vec[start:end+1].sum() != length:
                    if (end - start) >= (block_length // step_size):
                        n_blocks += 1
                        pos_info += f"_s{start * step_size}_e{(end) * step_size}"
                    vec[start:end+1] = False
                    break
    
    return n_bases, KaKs_r, n_blocks, pos_info

In [89]:
# combine kaks files 
C1G11_C1C5v2_KaKs_all_df_1 = pd.read_table("SNP_DP5/variant_seq/C1G11_C1C5v2split_114_6_1.axt.kaks", sep="\t", header=0)
C1G11_C1C5v2_KaKs_all_df_2 = pd.read_table("SNP_DP5/variant_seq/C1G11_C1C5v2split_114_6_2.axt.kaks", sep="\t", header=0)
C1G11_C1C5v2_KaKs_all_df_3 = pd.read_table("SNP_DP5/variant_seq/C1G11_C1C5v2split_114_6_3.axt.kaks", sep="\t", header=0)

In [90]:
C1G11_C1C5v2_KaKs_all_df_shaped = pd.concat([C1G11_C1C5v2_KaKs_all_df_1, C1G11_C1C5v2_KaKs_all_df_2, C1G11_C1C5v2_KaKs_all_df_3],ignore_index=True)

In [91]:
print(len(C1G11_C1C5v2_KaKs_all_df_1))
print(len(C1G11_C1C5v2_KaKs_all_df_2))
print(len(C1G11_C1C5v2_KaKs_all_df_3))
print(len(C1G11_C1C5v2_KaKs_all_df_shaped))

1249817
1249878
226553
2776362


In [92]:
#C1G11_C1C5v2_KaKs_all_df = pd.read_table("SNP_DP2/variant_seq/C1G11_C1C5v2split_114_6.axt.kaks", sep="\t", header=0)
print("finished reading data")
#C1G11_C1C5v2_KaKs_all_df_shaped = C1G11_C1C5v2_KaKs_all_df.copy()
C1G11_C1C5v2_KaKs_all_df_shaped['pos'] = C1G11_C1C5v2_KaKs_all_df_shaped['Sequence'].str.extract(r'\((\d+)\-', expand=False).astype(int)
C1G11_C1C5v2_KaKs_all_df_shaped['pair'] = C1G11_C1C5v2_KaKs_all_df_shaped['Sequence'].str.extract(r'.{11}(.*)\(', expand=False)
C1G11_C1C5v2_KaKs_all_df_shaped['geneid'] = C1G11_C1C5v2_KaKs_all_df_shaped['Sequence'].str.extract(r'(.{10})', expand=False)

geneid_vec = C1G11_C1C5v2_KaKs_all_df_shaped['geneid'].unique()
KaKs_all_df = pd.DataFrame({'geneid': geneid_vec})
KaKs_all_df['KaKs_bases'] = 0
KaKs_all_df['KaKs_r'] = 0
KaKs_all_df['n_blocks'] = 0
KaKs_all_df['pos_info'] = ''

KaKs_all_df2 = pd.DataFrame({'geneid': geneid_vec})
KaKs_all_df2['KaKs_bases2'] = 0
KaKs_all_df2['KaKs_r2'] = 0
KaKs_all_df2['n_blocks2'] = 0
KaKs_all_df2['pos_info2'] = ''

finished reading data


In [93]:
print("start calculating high KaKs region")
for idx, geneid in tenumerate(geneid_vec):
    geneid_df = C1G11_C1C5v2_KaKs_all_df_shaped[C1G11_C1C5v2_KaKs_all_df_shaped['geneid'] == geneid]
    res1 = get_KaKs_region(geneid_df)
    KaKs_all_df.loc[KaKs_all_df['geneid'] == geneid, ['KaKs_bases', 'KaKs_r', 'n_blocks', 'pos_info']] = res1
    res2 = get_KaKs_region(geneid_df, threshold=2)
    KaKs_all_df2.loc[KaKs_all_df2['geneid'] == geneid, ['KaKs_bases2', 'KaKs_r2', 'n_blocks2', 'pos_info2']] = res2
    #print(f"finished {geneid}")

KaKs_all_df_update = pd.merge(KaKs_all_df, KaKs_all_df2, on='geneid')

start calculating high KaKs region


  0%|          | 0/9695 [00:00<?, ?it/s]

/var/folders/s9/cpzp3wfd7831nz97c48532140000gn/T/ipykernel_75860/1754652586.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.006756756756756757' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  KaKs_all_df.loc[KaKs_all_df['geneid'] == geneid, ['KaKs_bases', 'KaKs_r', 'n_blocks', 'pos_info']] = res1
/var/folders/s9/cpzp3wfd7831nz97c48532140000gn/T/ipykernel_75860/1754652586.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0011299435028248588' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  KaKs_all_df2.loc[KaKs_all_df2['geneid'] == geneid, ['KaKs_bases2', 'KaKs_r2', 'n_blocks2', 'pos_info2']] = res2


In [94]:
KaKs_all_df_update.to_csv("SNP_DP5/C1G11_C1C5v2_KaKs_all.csv", index=False)